In [1]:
import numpy as np
import pandas as pd 
pd.set_option("max_columns",None)
from sklearn.linear_model import LinearRegression, Ridge

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_error,make_scorer,r2_score,mean_absolute_percentage_error
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import Lasso, Ridge, SGDRegressor,LinearRegression,RidgeCV,ElasticNet
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler,RobustScaler
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import ExtraTreeRegressor,DecisionTreeRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error,make_scorer,r2_score,mean_absolute_percentage_error,explained_variance_score

pd.set_option('max_columns', 200)



In [2]:
#df = pd.ExcelFile('C:/Users/halil/Desktop/soalr_data.xlsx').parse('sheet 1')
weather_raw = pd.read_excel('C:/Users/halil/Desktop/soalr_data.xlsx',sheet_name="weather")
guneko_raw = pd.read_excel('C:/Users/halil/Desktop/soalr_data.xlsx',sheet_name="1000255-GUNEKO")

In [4]:
guneko_production = guneko_raw[["Date","Production"]]
guneko_gti = guneko_raw[["Date.1","GTI"]]


weather_guneko = weather_raw.loc[weather_raw.name ==1000255]
ali = pd.merge(guneko_production,weather_guneko,left_on="Date",right_on="date")
dataset = ali.drop(["name","date","lat","lon"],axis=1)

In [29]:
conversion = dataset["Date"].astype(str)

dataset["year"] = conversion.apply(lambda x:int(x[:4]))
dataset["month"] = conversion.apply(lambda x:int(x[5:7]))
dataset["day"] = conversion.apply(lambda x:int(x[8:10]))
dataset["hour"] = conversion.apply(lambda x:int(x[11:13]))
onehot_encoded = dataset.drop(["Date"],axis=1)

In [66]:
a = onehot_encoded.loc[onehot_encoded["year"] == 2021]
b = a.loc[a["month"]== 12]
b

,Production,temperature,cloud cover,global_rad:W,diffuse_rad:W,direct_rad:W,relative_humidity_2m:p,wind_speed_10m:ms,prob_precip_1h:p,t_apparent:C,sun_elevation:d,year,month,day,hour
16404,0.000000,0.489286,0.703,0.355487,0.341203,0.241080,0.581493,0.438776,0.525532,0.499253,0.705333,2021,12,1,12
16405,0.091091,0.494643,0.366,0.369578,0.321907,0.267606,0.460568,0.326531,0.000000,0.500747,0.701333,2021,12,1,13
16406,0.425425,0.498214,0.810,0.327803,0.286947,0.236620,0.436383,0.255102,0.000000,0.497758,0.675333,2021,12,1,14
16407,0.501502,0.489286,0.193,0.246852,0.221112,0.175587,0.436383,0.173469,0.000000,0.482810,0.632000,2021,12,1,15
16408,0.506507,0.476786,0.155,0.129422,0.128490,0.085446,0.458465,0.132653,0.000000,0.460389,0.575333,2021,12,1,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17128,0.000000,0.319643,0.500,0.000000,0.000000,0.000000,0.722397,0.173469,0.000000,0.298954,0.374000,2021,12,31,19
17129,0.000000,0.326786,0.518,0.000000,0.000000,0.000000,0.687697,0.102041,0.000000,0.325859,0.295333,2021,12,31,20
17130,0.000000,0.325000,0.533,0.000000,0.000000,0.000000,0.689800,0.091837,0.000000,0.324365,0.216000,2021,12,31,21
17131,0.000000,0.307143,0.694,0.000000,0.000000,0.000000,0.762355,0.183673,0.000000,0.286996,0.137333,2021,12,31,22


In [53]:
def calculate_ridge(train_x,test_x,train_y,test_y):
    ridge_reg = Ridge(alpha=1) 

    ridge_reg.fit(train_x,train_y)

    ridge_train_1 = ridge_reg.predict(train_x)
    ridge_test_1 = ridge_reg.predict(test_x)

    return mean_squared_error(train_y, ridge_train_1), mean_squared_error(test_y, ridge_test_1)



def calculate_lasso(train_x_2,test_x_2,train_y_2,test_y_2):

    lasso_reg = Lasso(alpha=0.1)
    lasso_reg.fit(train_x_2,train_y_2)
    lasso_train_2 = lasso_reg.predict(train_x_2)
    lasso_test_2 = lasso_reg.predict(test_x_2)
    
    return mean_squared_error(train_y_2, lasso_train_2), mean_squared_error(test_y_2, lasso_test_2)

In [49]:

def metrics(y_train1,y_train_pred1,y_test1,y_test_pred1):
    print("for train data, mean squared error is",mean_squared_error(y_train1, y_train_pred1))
    print("for train data, mean absolute error is:",mean_absolute_error(y_train1, y_train_pred1))
    print("for train data, mean absolute percentage error is",mean_absolute_percentage_error(y_train1, y_train_pred1),"\n")

    print("for test data, mean squared error is",mean_squared_error(y_test1, y_test_pred1))
    print("for test data, mean absolute error is:",mean_absolute_error(y_test1, y_test_pred1))
    print("for test data, mean absolute percentage error is",mean_absolute_percentage_error(y_test1, y_test_pred1),"\n")


In [41]:
#normalization in range (-1,1)
def normalize(df,column_list):
    for col in column_list:
        feature_range = (0,1)
        min_max_scaler = MinMaxScaler(feature_range=feature_range)

        df[col] = min_max_scaler.fit_transform(df[col].values.reshape(-1,1))

    return df

In [28]:
def get_year(whole_data,target_year):

    whole_data = whole_data.loc[whole_data.year== target_year]
    whole_data = whole_data.drop(["year"],axis=1)
    whole_data = whole_data.reset_index(drop=True)

    return whole_data

In [44]:
def make_data(dataframe):
    train = dataframe[:len(dataframe)- 24*5]
    test = dataframe[len(dataframe)- 24*5:]

    x_train = train.copy()
    x_test = test.copy()

    y_train = x_train.pop('Production')
    y_test = x_test.pop('Production')

    return x_train, x_test, y_train, y_test 

In [79]:
def get_months(target_data):
    months = []

    for i in range(1,13):
        month_data = target_data.loc[target_data["month"]==i].copy()
        aw1 = month_data.reset_index(drop=True)
        aw1 = aw1.drop(["month"],axis=1)
        months.append(aw1)

    return months

In [80]:
normalized = normalize(onehot_encoded,onehot_encoded.columns[0:11])
data_2021 = get_year(normalized,2021)
data_months = get_months(data_2021)


In [45]:
x_train, x_test, y_train, y_test = make_data(data_months[3])

In [107]:
def calculate_errors(month_data):
    errors = []
    for i in range(0,12):
        print(month_data[i])
        #x_train, x_test, y_train, y_test = make_data(month_data.loc[month_data["month"]==i])
        x_train, x_test, y_train, y_test = make_data(month_data[i])
        ridge = calculate_ridge(x_train, x_test, y_train, y_test)

        lasso = calculate_lasso(x_train, x_test, y_train, y_test)

        errors.append({f"ridge":ridge,"lasso":lasso})

    return errors

In [ ]:
errors = calculate_errors(data_months)

In [110]:
ridge_train =[]
ridge_test = []
lasso_train = []
lasso_test = []
for i in errors:
    ridge_train.append(i["ridge"][0])
    ridge_test.append(i["ridge"][1])
    lasso_train.append(i["lasso"][0])
    lasso_test.append(i["lasso"][1])

aylar = ["January","February","March","April","May","June","July","August","September","October","November","December"]

seri1  = pd.Series(ridge_train) 
seri2  = pd.Series(ridge_test) 
seri3  = pd.Series(lasso_train) 
seri4  = pd.Series(lasso_test) 
seri5 = pd.Series(aylar)

frame = {"months":seri5,"ridge_train":ridge_train, "ridge_test":ridge_test,"lasso_train":lasso_train,"lasso_test":lasso_test}
result = pd.DataFrame(frame)

In [112]:
data = result

import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Bar(x=data["months"],y=data["ridge_test"],name='Ridge Model Train Results', marker_color='indianred'))

fig.add_trace(go.Bar(x=data["months"], y=data["ridge_test"], name='Ridge Model Test Results',marker_color='lightsalmon'))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [113]:

fig = go.Figure()
fig.add_trace(go.Bar(x=data["months"],y=data["lasso_test"],name='Lasso Model Train Results', marker_color='indianred'))

fig.add_trace(go.Bar(x=data["months"], y=data["lasso_test"], name='Lasso Model Test Results',marker_color='lightsalmon'))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()